Given a sample

In [1]:
sample = {"review": "نسبت به قیمت جدیدش قهوه های بهتری رو توی این رنج قیمت میشه خرید", "output": "NEGATIVE"}

# Prepare input prompt

In [2]:
import prompt

In [3]:
sample_prompt = prompt.ENGLISH_ONE.format(review=sample["review"])

In [4]:
print(sample_prompt)

The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it.
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER.
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses.

Examples:

Example 1:
Review: طعم بد . بوی بد . فقط قیمتش خوبه .
Sentiment: NEGATIVE

Example 2:
Review: با بقیه چایی ها هیچ فرقی نداره چه طعم چه رنگ
Sentiment: NEUTRAL

Example 3:
Review: خوب بود و در باز بودن و طعم خوبی داشت
Sentiment: POSITIVE

Example 4:
Review: مزه اش یه جور خاصه تند تند نیس ولی متفاوته
Sentiment: OTHER

Review: نسبت به قیمت جدیدش قهوه های بهتری رو توی این رنج قیمت میشه خرید
Sentiment: 



# Evaluate model's response

In [5]:
sample_GPT_response = """Negative"""

In [6]:
acceptable_classes = [
    "POSITIVE",
    "NEGATIVE",
    "NEUTRAL",
    "OTHER"
]

In [7]:
def post_proccess_response(response):
    response = response.upper()
    for label in acceptable_classes:
        if label in response:
            return label
    return "OTHER"

In [8]:
proccessed_response = post_proccess_response(sample_GPT_response)
proccessed_response

'NEGATIVE'

In [9]:
from sklearn.metrics import classification_report

actual = [sample["output"]]
predicted = [proccessed_response]
print(classification_report(actual, predicted, zero_division=0, target_names=acceptable_classes, labels=acceptable_classes))

              precision    recall  f1-score   support

    POSITIVE       0.00      0.00      0.00         0
    NEGATIVE       1.00      1.00      1.00         1
     NEUTRAL       0.00      0.00      0.00         0
       OTHER       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00         1
   macro avg       0.25      0.25      0.25         1
weighted avg       1.00      1.00      1.00         1



In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter
import pandas as pd
import numpy as np

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=0)
    precision = precision_score(actual, predicted, average='weighted', zero_division=0)
    recall = recall_score(actual, predicted, average='weighted', zero_division=0)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = list(set(actual + predicted))
    classes.append('All Classes')
    metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

    probability_table = pd.DataFrame(index=metrics, columns=classes)

    for class_label in classes:
        if class_label == 'All Classes':
            probability_table[class_label]["F1 Score"] = f1
            probability_table[class_label]["Precision"] = precision
            probability_table[class_label]["Recall"] = recall
        else:
            class_precision = precision_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_recall = recall_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_f1 = f1_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            probability_table[class_label]["F1 Score"] = class_f1
            probability_table[class_label]["Precision"] = class_precision
            probability_table[class_label]["Recall"] = class_recall
    probability_table.fillna(value="-", inplace = True)
    probability_table['All Classes']["Accuracy"] = accuracy
    return probability_table

In [11]:
results = calculate_metrics([sample["output"]], [proccessed_response])
results

,NEGATIVE,All Classes
Accuracy,-,1.0
F1 Score,[1.0],1.0
Precision,[1.0],1.0
Recall,[1.0],1.0
